In [ ]:
import json

In [ ]:
[x.delete() for x in Manuscript.objects.filter(lib_code=None)]

In [ ]:
items = MsImage.objects.filter(manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    ms = Manuscript.objects.get(legacy_pk=ms_id)
    x.manuscript = ms
    x.save()
        

In [ ]:
items = MsPart.objects.filter(part_of_manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    try:
        ms = Manuscript.objects.get(legacy_pk=ms_id)
    except:
        print(ms_id, x.id)
        continue
    x.part_of_manuscript = ms
    x.save()